In [1]:
%%time
import manager_dataset as MD
import question_processing as QP
import information_retrieval as IR
import named_entity_recognition as NER
import answer_processing as AP
from gensim.models import KeyedVectors
from util import util

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 31 s


### Settings

Questions recive all the questions treated

In [2]:
questions = MD.questions(treated=True)
incosiderate_classes = ['X', 'MANNER', 'OBJECT', 'OTHER', 'DEFINITION']
questions = QP.remove_incosiderate_classes(questions, incosiderate_classes)  # Question with incosiderate class is removed

The questions that has an answer will be used for test while another questions will be used for train.

In [3]:
train_questions, test_questions = MD.split_questions(questions)

## Question Processing

### Question Classification

Questions to train and test

In [4]:
qc_X_train, qc_y_train = QP.separete_questions(train_questions)

Or Load the Word2Vec model

In [ ]:
%%time
w2v_patch = 'data/word_embedding/cbow_s300.txt'
#w2v_patch = 'data/word_embedding/cbow_s50.txt'  # Fast test
w2v = KeyedVectors.load_word2vec_format(w2v_patch, unicode_errors="ignore")
model_w2v = {w: vec for w, vec in zip(w2v.index2word, w2v.syn0)}

And Train a SVM model

In [ ]:
%%time
classifier = QP.svm_classifier()
vectorizer = QP.HybridVectorizer(model_w2v)
#vectorizer = QP.MeanEmbeddingVectorizer(model_w2v)
#vectorizer = QP.TfidfHybridVectorizer(model_w2v)
#vectorizer = QP.SequenceHybridVectorizer(model_w2v)
#vectorizer = QP.count_vectorizer(model_w2v)
#vectorizer = QP.tfidf_vectorizer(model_w2v)
model = QP.train_model(qc_X_train, qc_y_train, classifier, vectorizer)

Or Loading the the trained SVM model

In [5]:
%%time
import dill
model = dill.load(open(QP.answer_type_file, 'rb'))

Wall time: 23.4 s


Answer type is predicted for all questions

In [6]:
questions = QP.predict_answer_type(model, test_questions)

#### Testing

In [ ]:
qc_X_test, qc_y_test = QP.separete_questions(test_questions)
print('Size train:',len(qc_y_train),'  Size test:',len(qc_y_test))

In [ ]:
%matplotlib notebook
confusion_matrix = QP.testing(model, qc_X_test, qc_y_test)
QP.plot_confusion_matrix(confusion_matrix, ['LOCATION', 'MEASURE', 'ORGANIZATION', 'PERSON', 'TIME'], normalize=True)

### Query Formulation

All test questions will recive the query for IR module. 

In [7]:
test_questions = QP.queryFormulation(test_questions)

## Information Retrieval

### Document Retrieval

In [8]:
import information_retrieval as IR
ir = IR.InformationRetrieval()

Start the IR module. If the documents was not indexed then they will be.
When reset is True, all indexed documents is removed and indexed again.

In [9]:
ir.start(reset=False)

In [10]:
%%time
test_questions = ir.retrievalDocuments(test_questions)

Wall time: 12.7 s


#### Testing

In [11]:
import numpy as np

precisions = []
recalls = []
f_scores = []
b = 0.5
for question in test_questions:
    relevants = []
    for answer in question['answers']:
        if answer['doc'] is not None and len(answer['doc']) > 0:
            relevants.append(answer['doc'])
    total_relevants = len(relevants)
    if total_relevants == 0:
        continue
    relevents_retrieval = 0
    for retrieval in question['retrieval']:
        if retrieval in relevants:
            relevents_retrieval += 1
    if len(question['retrieval']) == 0:
           # print(question['query'])
            precision = 0
    else:
        precision = relevents_retrieval / len(question['retrieval'])
    precisions.append(precision)
    recall = relevents_retrieval / total_relevants
    recalls.append(recall)
    if (precision + recall) == 0:
        f_score = 0
    else:
        f_score = 2 * ((precision * recall) / (precision + recall))
    f_scores.append(f_score)

print('Precision: '+str(np.mean(precisions)))
print('Recall: '+str(np.mean(recalls)))
print('F-Score: '+str(np.mean(f_scores)))

Precision: 0.060141509434
Recall: 0.492391060198
F-Score: 0.104694452329


### Named Entity Recognition

In [12]:
import named_entity_recognition as NER

model_ner = NER.train()

#### Testing

In [ ]:
NER.predict(model_ner, u'Em Porto Seguro mora Maria')

### Passage Retrieval

In [13]:
%%time
IR.retrievalPassagesQuestions(test_questions, model_ner, ir)

Wall time: 1min 15s


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
test_questions[0]['answer_candidates'][1]

## Answer Processing

### Answer Extraction

In [20]:
test_questions = AP.answer_candidates(test_questions, QP, ir, NER, model_ner)

[ . . . . . . . . . . ]


### Answer Extraction

In [78]:
%%time
importlib.reload(AP)
test_questions = AP.finals_answer(test_questions)

[ . . . . . . . . . . ]
Wall time: 194 ms


In [77]:
total = len(test_questions)
right = 0

cc_total = 0  # Total of question with correct candidates
cc_right = 0

for question in test_questions:
    aux = False
    if question['correct_answers_candidates']:
        aux = True
        cc_total += 1
    for answer in question['answers']:
        if answer['answer'] is not None:
            a1 = answer['answer'].lower().replace(',', '').replace('.', '').replace(';', '').replace('(','').replace(')','').replace(u'«','').replace(u'»','').strip()
            a2 = question['final_answer'].lower().replace(',', '').replace('.', '').replace(';', '').replace('(','').replace(')','').replace(u'«','').replace(u'»','').strip()
            if a1 in a2 or a2 in a1:
                question['correct_final_answer'] = True
                right += 1
                if aux:
                    cc_right += 1
                break
print(str(total) + ' / ' + str(right))
print('Accuracy: '+ '%.3f' % ((right/total)*100)+' %')
print('\nCorrect answer candidates:\n' + str(cc_total) + ' / ' + str(cc_right))
print('Accuracy: '+ '%.3f' % ((cc_right/cc_total)*100)+' %')

655 / 91
Accuracy: 13.893 %

Correct answer candidates:
389 / 91
Accuracy: 23.393 %


#### Testing

Check if the answer_candidates have the right answer

In [41]:
AP.test_answer_candidates(test_questions)

655 / 389
Total recall: 59.389 %

Correct answer type:
594 / 365
Recall: 61.448 %


## Final Evaluate

In [39]:
for i in range(len(test_questions)):
    print(test_questions[i]['question'])
    print(test_questions[i]['final_answer'])
    for answer in test_questions[i]['answers']:
        if answer['answer'] is not None:
            print(answer['answer'].strip(), end=' | ')
    print('\n')

Em que cidade se encontra a prisão de San Vittore?
Milão
Milão | 

Onde era o campo de concentração de Auschwitz?
Europa Ocidental
Sul da Polónia | Polónia | Sudoeste da Polónia ocupada | 

Quem foi o autor de Mein Kampf?
presidente Bill Clinton
Adolf Hitler | 

Qual é a capital da Rússia?
Budinovsk
Moscovo | 

Quem foi o primeiro presidente dos Estados Unidos?
Lyndon Johnson
George Washington | 

Com quem se casou Michael Jackson?
Hugo Francisco Alvarez Perez
Lisa Marie Presley | 

Quantos crimes são atribuídos ao Monstro de Florença?
22
16 assassinatos | 

Quantos desempregados há na Europa?
90 mil
23 milhões | 

Quantas religiões monoteístas há no mundo?
632
três | 

Nomeie uma pessoa acusada de pedofilia.
Associação Internacional de Lésbicas e Gays
Brendan Smyth | Hans Hermann Groer | 

Quem escreveu Ulisses?
PÚBLICO
James Joyce | 

Onde se situa o CERN?
Japão
Genebra | 

Em que dia calha o solstício de verão?
2 horas
aproximadamente a 21 de Junho | 

Onde fica o Museu do Hermitage

52 anos
731 | 

A que velocidade viaja a luz?
330 metros
300 mil quilómetros por segundo | 

Quando foram criadas as FPLM (Forças Populares de Libertação de Moçambique)?
1964
há 30 anos | 

Quando foi a independência de Cabo Verde?
25 de Abril
5 de Julho de 1975 | 

Quando estreia o filme Lisbon Story?
24 horas
16 de Dezembro | 

Quando foi aprovada a Declaração Universal dos Direitos do Homem?
10 de Dezembro de 1948
1948 | 

Quando morreu Salvador Allende?
169
11 de Setembro de 1973 | 

Quando morreu Simão Bolívar?
Fevereiro de 1992
17 de Dezembro de 1830 | 

Em que dia se comemora a independência do Brasil?
7 de Setembro
7 de Setembro | 

Quando se tornou A Portuguesa hino nacional?
1975
5 de Outubro de 1910 | 

Em que ano ocorreu o 25 de Abril?
10h55
1974 | 

Que país é campeão do mundo de futebol?
Portugal
Brasil | 

Onde é que caiu um meteorito em 1908?
Amazónia
Sibéria | Sibéria | 

Quantos capacetes azuis holandeses havia em Srebrenica, na Bósnia, em Julho de 1995?
cerca de 30 m

cerca de 20 km
135 quilômetros | 

Que quantidade de peixe come uma foca por dia?
cerca de 20 anos
20,25 quilos | 30 quilos | 25 quilos | 

Que partido regressou ao poder em Outubro de 1993?
Hospital Onassis
PASOK | 

De que jornal é repórter Clark Kent?
Reportagem Local Lois Lane
Daily Planet | Daily Planet | 

De que partido é líder António Guterres?
Câmara de Ovar Armando França
PS | Partido Socialista | Partido Socialista | 

Em que organização trabalha Luc Montagnier?
Reportagem Local Dez
Instituto Pasteur | Instituto Pasteur | 

Qual a banda de Siouxie Sioux?
Reportagem Local
Banshees | 

Qual era o maior partido italiano?
Partido Socialista PSI Bettino Craxi
Democracia Cristã | 

Qual o partido-irmão da CDU na Baviera?
50 por cento
União Social Cristã ( CSU ) | 

Que partido fundou Manfred Brunner?
PARTIDO ALEMÃO
Liga dos Cidadãos Livres ( BFB ) | 

Que time se mudou para Salt Lake City?
Junho
New Orleans Jazz | 

Quem assassinou Salvo Lima?
Carlo Alberto Dalla Chiesa
Máfia | 



no autódromo de Ímola , na Itália | circuito Enzo e Dino Ferrari | circuito de Ímola ( Itália ) | circuito de Monza , em Itália | Imola ( Itália ) | 

Em que país foi aprovada uma lei que proíbe a publicidade ao tabaco?
Código da Publicidade de 1990
França | 

Onde ocorreram casos de febre aftosa?
São Jorge do Patrocínio
Grécia | Catolé do Rocha , Brejo do Cruz , São Bento , Paulista , Belém do Brejo do Cruz e Brejo dos Santos | interior da Paraíba | São Jorge do Patrocínio , no Paraná | 

Quando aderirão à União Europeia a Áustria, a Suécia e a Finlândia?
64
1 de Janeiro de 1995 | 

Que estúdio produziu o filme Toy Story?
Centro de Computação Gráfica do Fraunhofer Institute
Pixar | Pixar | estúdios Disney | 

Quem era o capitão do Ajax em 1994?
Ajax Danny Blind
Danny Blind | 

Em que cidade é que Nietzsche escreveu Ecce Homo?
Rõcken
NIL | 

Quando viveu Ludwig Wittgenstein?
29 de Abril de 1951 Wittgenstein
1889-1951 | 

Quem foi votado o futebolista do ano na Bélgica em 1995?
George W

E quem venceu em 1994?
Ricardo Gandour
Chris Boardman | 

Com quem se casou a rainha Cristina?
George
 | 

Que idade tinha ela quando morreu?
13
63 anos | 

Quem foi o 13º rei de Portugal?
D Leonor
D. João II | 

Que empresa editou a Diciopédia?
Conselho Federal de Medicina
 | 

Que dinastia reinava em Portugal em 1500?
República Popular da China Xizang
Dinastia de Avis | 

Quantos são eles?
13 anos
quatro | 

Qual deles fica na Europa?
Mais 3000
 | 

Em que zona de Espanha nasce o rio Douro?
Primavera
na província de Sória, nos picos da Serra de Urbião (Sierra de Urbión) | 

Onde fica o parque Eduardo VII?
Câmara
centro de Lisboa | 

Quantos focos tem uma elipse?
6
dois | 

Quantos pontos vale um ensaio no râguebi?
50 e 65 minutos
cinco | 

Diga uma escritora sarda.
J S -- Xota
Grazia Deledda | 

Qual a freguesia mais populosa de Évora?
Queluz Sintra
Malagueira | 

Que equipa é treinada por Fernando Santos?
Botafogo
Sport Lisboa e Benfica | 

Que equipa era treinada por Fernando Santo

Nicolas Sarkozy | 

Qual a capital da Picardia?
Londres
Amiens | 

Qual é a capital do estado de Nova York?
Brasília
Albany | 

Quais são as províncias da Irlanda?
Reportagem Local
Leinster, Munster, Connacht e Ulster. | 

Que papa sucedeu a Leão X?
Falecido Júlio II Roma
Clement VII | 

Que países abrange a Lapónia?
Maxus
Noruega, Suécia, Finlândia e Federação Russa | 

De que país é originário?
Brasília
Grécia | 

Em que ano foi construída a sinagoga de Curação?
1888
1732 | 

Com que idade o Mequinho foi campeão brasileiro de xadrez?
14
13 anos | 

Quem dirigiu o Japão durante a Segunda Guerra Mundial?
Nagisa Oshima
Hirohito | 

Quantas repúblicas formavam a URSS?
15
15 | 

Em que país fica a Ossétia do Norte?
Aeroflot
Federação Russa | 

E a Ossétia do Sul?
ministério da Defesa
Geórgia | 

Qual a largura do Canal da Mancha no seu ponto mais estreito?
2000 CV 128 metros
34 km | 

Quem criou Descobridores de Catan?
Descobridores
Klaus Teuber | 

Quem é o santo patrono dos cervejeiros?

In [44]:
for c in test_questions[0]['answer_candidates']:
    print(c['score'], c['votes'])

0.0 0.0
1.0 1.0
0.038461538461538464 0.038461538461538464
0.23076923076923075 0.23076923076923075
0.9038461538461539 0.9038461538461539
0.8076923076923077 0.8076923076923077
0.5192307692307693 0.5192307692307693
0.038461538461538464 0.038461538461538464
0.038461538461538464 0.038461538461538464
0.038461538461538464 0.038461538461538464
0.038461538461538464 0.038461538461538464
0.4230769230769231 0.4230769230769231
0.038461538461538464 0.038461538461538464
0.038461538461538464 0.038461538461538464
0.1346153846153846 0.1346153846153846
0.038461538461538464 0.038461538461538464
0.7115384615384616 0.7115384615384616
0.038461538461538464 0.038461538461538464
0.23076923076923075 0.23076923076923075
0.6153846153846154 0.6153846153846154
0.5192307692307693 0.5192307692307693
0.3269230769230769 0.3269230769230769
0.1346153846153846 0.1346153846153846
0.038461538461538464 0.038461538461538464
0.4230769230769231 0.4230769230769231
0.3269230769230769 0.3269230769230769
0.038461538461538464 0.03846

In [64]:
sorted(test_questions[0]['answer_candidates'], key=lambda k: k['score'])[-1]

{'distance': 0,
 'doc_rank': 0.1,
 'full_answer': 'Itália',
 'passage_text': 'Milão Tangentópolis, a devoradora Da nossa enviada Alexandra Prado Coelho, em Milão A cidade mais rica e mais cara da Itália transformou-se no símbolo da corrupção: Milão -- Tangentópolis.',
 'qw_in_passage': 0.5,
 'score': 1.0,
 'votes': 1.0,
 'words': [('Itália', 20)]}

## Util

### Detailed Question Analysis

In [ ]:
question = {
    'question': u'Quem é o presidente da França?',
    'class': 'PERSON'
}

#question = test_questions[4]

print('Question: ' + question['question'])

# Question Classification
QP.predict_answer_type(model, [question])
print('Answer Type: ' + question['predict_class'])

# Query Formulation
QP.queryFormulation([question])
print('Query: ' + question['query'])

# Document Retrieval
ir.retrievalDocuments([question])
print('Number of retrieval documents: ' + str(len(question['retrieval'])))

# Answers Candidates
AP.answer_candidates([question], QP, ir, NER, model_ner, loading=False)
print('Number of answer candidates: ' + str(len(question['answer_candidates'])))

# Final Answer
AP.finals_answer([question])
print('Answer: ' + question['final_answer'])

Show documents texts

In [ ]:
for doc_id in question['retrieval']:
    print('\n'+doc_id+'\n'+ir.documentText(doc_id))

Show answers candidates

In [ ]:
    for ac in question['answer_candidates']:
        print(ac['full_answer'])

Show most vote answer candidates

In [ ]:
import operator
votes = {}
for candidate in question['answer_candidates']:
    if candidate['full_answer'].lower() not in question['question'].lower():
        if candidate['full_answer'] not in votes:
            votes[candidate['full_answer']] = candidate['votes']
[(k, votes[k]) for k in sorted(votes, key=votes.get, reverse=True)]

### Others

In [ ]:
ir.stop()

In [74]:
import importlib
importlib.reload(AP)
#raise IpyExit

<module 'answer_processing' from 'C:\\Users\\Eduardo\\UFRGS\\QA\\qa-chave\\answer_processing.py'>